In [ ]:
from itertools import islice, cycle
from tau_profile_parser import TauProfileParser
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px


In [ ]:
path_to_multi_files = 'path/to/files'
multi_files = TauProfileParser.parse(path_to_multi_files,MULTI=True)

In [ ]:
time_sorted = multi_files.interval_data().sort_values(by=[('Exclusive','TIME')], ascending=False)

In [ ]:
time_sorted[time_sorted['Timer Name'].str.match('OpenMP_Implicit_Task')]

### Removing Timers
I removed OpenMP_Thread_Type_ompt_thread_worker, .TAU application, and OpenMP_Implicity_Task from the data so that we can just work with smaller functions. 
I used a method of just taking 2 sub DF's from the time_sorted, concat them together and then drop_duplicates(). The logic for finding which lines is demonstrated above. This was an example I found on stack overflow: https://stackoverflow.com/questions/37313691/how-to-remove-a-pandas-dataframe-from-another-dataframe combined with this example: https://davidhamann.de/2017/06/26/pandas-select-elements-by-string/ .



In [ ]:
time_sorted = pd.concat([time_sorted,time_sorted[time_sorted['Timer Name'].str.match('OpenMP_Implicit_Task')],time_sorted[time_sorted['Timer Name'].str.match('OpenMP_Implicit_Task')]]).drop_duplicates(keep=False)

In [ ]:
time_sorted = pd.concat([time_sorted,time_sorted[time_sorted['Timer Name'].str.match('.TAU application')],time_sorted[time_sorted['Timer Name'].str.match('.TAU application')]]).drop_duplicates(keep=False)

In [ ]:
time_sorted = pd.concat([time_sorted,time_sorted[time_sorted['Timer Name'].str.match('OpenMP_Thread_Type_ompt_thread_worker')],time_sorted[time_sorted['Timer Name'].str.match('OpenMP_Thread_Type_ompt_thread_worker')]]).drop_duplicates(keep=False)

# Selecting Columns for Waffle Plot
We can select any of the columns we would like to analyze. Waffle plot would only be useful for a few Items at best.

In [ ]:
import matplotlib.pyplot as plt 
from pywaffle import Waffle

In [ ]:
slimmed = pd.concat([time_sorted[['Timer Name']],time_sorted[('Exclusive','TIME')]], axis=1)
slimmed_wo_index = slimmed.reset_index()
slim_timer = slimmed_wo_index['Timer Name']
slim_TIME = slimmed_wo_index[('Exclusive', 'TIME')]


In [ ]:
slim_timer_time = pd.concat([slimmed_wo_index['Timer Name'],slimmed_wo_index[('Exclusive', 'TIME')]],axis=1)
slim_timer_time.head(10)

#### We want the (Exclusive,TIME) column to be a smaller amount to compare them.

In [ ]:
reducing_factor = 10000
slim_timer_time[('Exclusive','TIME')] = slim_timer_time[('Exclusive','TIME')].apply(lambda x: x/reducing_factor)


Calculating the `other` if we wanted to include it for comparison of functions, but it is so big that it would drown out our analysis of the top functions.


In [ ]:
other = slim_timer_time[5:].sum()


### Waffle Plot
The library pyWaffle can be downloaded using `pip install pywaffle`. Some other examples are found on their GitHub: https://github.com/gyli/PyWaffle. Waffle plots get increasingly difficult to read when there are more than a few elements in the plot. I selected the top five functions based on time to demonstrate. 

In [ ]:
fig = plt.figure( 
    FigureClass = Waffle, 
    rows = 3, 
    values = slim_timer_time[('Exclusive','TIME')][:5], 
    labels = list(slim_timer_time['Timer Name'][:5]) ,
    legend={'loc': 'best','bbox_to_anchor': (2.7,1.1)},
    title={'label': 'Time for top 5 functions', 'loc': 'left'},
) 